In [1]:
# =============================================================================
# File: seq2seq_multi_step_demand_forecasting.py
# Topic: Sequence-to-Sequence (Encoder-Decoder) for multi-step demand forecasting
# Input: Panel time series (sku_id, location_id, date, demand) + optional metadata
# Output: H-step demand forecast curve + replenishment suggestions (ROP / order qty)
# =============================================================================

from __future__ import annotations

import math
from dataclasses import dataclass
from typing import Dict, Tuple, Optional

import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# =============================================================================
# 0. TOP-LEVEL CONFIG AND CONSTANTS
# =============================================================================

RANDOM_SEED = 42

# Time-series windowing
LOOKBACK_DAYS = 28          # encoder length (history)
FORECAST_HORIZON = 14       # decoder length (future curve)

# Training
BATCH_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 1e-3

# Model
ENCODER_UNITS = 64
DECODER_UNITS = 64
DROPOUT = 0.1
EMBED_DIM = 16

# Replenishment / Inventory policy (toy but realistic structure)
SERVICE_LEVEL = 0.95        # used for safety stock approximation
REVIEW_PERIOD_DAYS = 1      # daily review policy (s, S) simplified
MIN_ORDER_QTY = 0           # can set to supplier MOQ if you have one

# Column names (keep consistent across course files)
COL_DATE = "date"
COL_SKU = "sku_id"
COL_LOC = "location_id"
COL_DEMAND = "demand"

# Reproducibility
tf.keras.utils.set_random_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


In [3]:
# =============================================================================
# 1. UTILITIES: DATA SIMULATION (FOR A SELF-CONTAINED DEMO)
# =============================================================================

def simulate_sku_location_demand(
    n_skus: int = 50,
    n_locations: int = 3,
    n_days: int = 365,
    cold_start_ratio: float = 0.2,
) -> pd.DataFrame:
    """
    Create a realistic-enough panel time series with:
    - weekday seasonality
    - trend
    - random promo pulses
    - SKU/Location effects
    - cold-start SKUs that appear later in the calendar

    Why this matters:
    - Forecasting in SCM is rarely "one series"; it's many series (SKU x location).
    - Cold-start is common (new products, new DCs, assortment changes).
    """
    start = pd.Timestamp("2024-01-01")
    dates = pd.date_range(start, periods=n_days, freq="D")

    sku_ids = [f"SKU_{i:04d}" for i in range(n_skus)]
    loc_ids = [f"LOC_{j:02d}" for j in range(n_locations)]

    # Which SKUs are cold-start: they start selling later
    n_cold = int(round(n_skus * cold_start_ratio))
    cold_skus = set(np.random.choice(sku_ids, size=n_cold, replace=False))

    rows = []
    for sku in sku_ids:
        # SKU baseline and volatility
        sku_level = np.random.uniform(5, 60)
        sku_noise = np.random.uniform(0.8, 2.5)

        # Cold-start: sales begin later (e.g., after product launch)
        launch_offset = np.random.randint(60, 200) if sku in cold_skus else 0

        for loc in loc_ids:
            loc_multiplier = np.random.uniform(0.7, 1.3)

            # Promo pulses: rare multiplicative spikes
            promo_days = set(np.random.choice(np.arange(n_days), size=n_days // 25, replace=False))

            for t, d in enumerate(dates):
                if t < launch_offset:
                    demand = 0.0
                else:
                    weekday = d.dayofweek  # 0=Mon ... 6=Sun
                    weekly_season = 1.0 + 0.2 * math.sin(2 * math.pi * weekday / 7)

                    trend = 1.0 + 0.0008 * t
                    promo = 1.0 + (np.random.uniform(0.3, 1.2) if t in promo_days else 0.0)

                    mean = sku_level * loc_multiplier * weekly_season * trend * promo
                    demand = np.random.poisson(lam=max(mean, 0.1)) + np.random.normal(0, sku_noise)

                    # Demand must be non-negative for most retail/parts use-cases
                    demand = float(max(demand, 0.0))

                rows.append(
                    {
                        COL_DATE: d,
                        COL_SKU: sku,
                        COL_LOC: loc,
                        COL_DEMAND: demand,
                    }
                )

    df = pd.DataFrame(rows).sort_values([COL_SKU, COL_LOC, COL_DATE]).reset_index(drop=True)
    return df

In [4]:
# =============================================================================
# 2. FEATURE ENGINEERING & WINDOWING
# =============================================================================

def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add simple calendar features.

    Why this matters:
    - Seq2Seq can learn seasonality implicitly, but giving explicit calendar
      signals often improves generalization (especially with short histories).
    """
    out = df.copy()
    out["dow"] = out[COL_DATE].dt.dayofweek.astype(np.int32)
    out["month"] = out[COL_DATE].dt.month.astype(np.int32)
    out["is_weekend"] = (out["dow"] >= 5).astype(np.int32)
    return out


def fit_scaler_per_series(df: pd.DataFrame) -> Dict[Tuple[str, str], Tuple[float, float]]:
    """
    Per-(SKU,Location) scaling using mean/std.

    Why per-series:
    - SKU demand scales differ massively (fast vs slow movers).
    - Global scaling often underfits slow movers and overfits fast movers.
    """
    stats = {}
    grp = df.groupby([COL_SKU, COL_LOC])[COL_DEMAND]
    for key, s in grp:
        mu = float(s.mean())
        sigma = float(s.std(ddof=0))
        sigma = sigma if sigma > 1e-6 else 1.0
        stats[key] = (mu, sigma)
    return stats


def apply_scaler(
    df: pd.DataFrame,
    stats: Dict[Tuple[str, str], Tuple[float, float]],
) -> pd.DataFrame:
    """
    Apply per-series scaling. Result column: demand_scaled.

    Note:
    - For cold-start windows with all zeros, mean/std are stable.
    """
    out = df.copy()
    mu = []
    sig = []
    for sku, loc in zip(out[COL_SKU], out[COL_LOC]):
        m, s = stats[(sku, loc)]
        mu.append(m)
        sig.append(s)
    out["mu"] = np.array(mu, dtype=np.float32)
    out["sigma"] = np.array(sig, dtype=np.float32)
    out["demand_scaled"] = (out[COL_DEMAND].astype(np.float32) - out["mu"]) / out["sigma"]
    return out


def build_id_maps(df: pd.DataFrame) -> Tuple[Dict[str, int], Dict[str, int]]:
    """Create integer IDs for embeddings."""
    sku_map = {v: i for i, v in enumerate(sorted(df[COL_SKU].unique()))}
    loc_map = {v: i for i, v in enumerate(sorted(df[COL_LOC].unique()))}
    return sku_map, loc_map


@dataclass
class WindowedDataset:
    """
    Holds arrays for a teacher-forced seq2seq training setup.

    Encoder inputs:
      - past demand_scaled
      - time features (dow, month, weekend)
      - embeddings IDs (sku_id, location_id) as separate inputs

    Decoder inputs (teacher forcing):
      - previous true demand_scaled values, shifted right by 1 step
      - future time features (optional but helpful)

    Decoder target:
      - true demand_scaled for horizon
    """
    enc_demand: np.ndarray
    enc_time: np.ndarray
    dec_in_demand: np.ndarray
    dec_time: np.ndarray
    y: np.ndarray
    sku_id: np.ndarray
    loc_id: np.ndarray


def make_seq2seq_windows(
    df: pd.DataFrame,
    sku_map: Dict[str, int],
    loc_map: Dict[str, int],
    lookback: int = LOOKBACK_DAYS,
    horizon: int = FORECAST_HORIZON,
    step: int = 1,
) -> WindowedDataset:
    """
    Convert panel data into (encoder_window, decoder_window) samples.

    Key design choices:
    - Teacher forcing requires decoder inputs. We feed the *previous* true value.
    - We include time features on both encoder and decoder sides.
    - We attach sku_id and location_id to every sample (embeddings).

    Cold-start handling:
    - Cold-start shows up naturally as early sequences with many zeros.
    - Embeddings let the model learn "SKU identity priors" and "location priors"
      that can help even when history is short.
    """
    enc_demand_list, enc_time_list = [], []
    dec_in_demand_list, dec_time_list = [], []
    y_list = []
    sku_id_list, loc_id_list = [], []

    # Ensure correct ordering
    df = df.sort_values([COL_SKU, COL_LOC, COL_DATE]).reset_index(drop=True)

    # Time features we will feed to model
    time_cols = ["dow", "month", "is_weekend"]

    for (sku, loc), g in df.groupby([COL_SKU, COL_LOC], sort=False):
        g = g.reset_index(drop=True)

        demand = g["demand_scaled"].to_numpy(dtype=np.float32)
        time_feat = g[time_cols].to_numpy(dtype=np.float32)

        n = len(g)
        # windows: [t-lookback, t) -> forecast [t, t+horizon)
        for t in range(lookback, n - horizon + 1, step):
            enc_d = demand[t - lookback : t]                          # (lookback,)
            enc_t = time_feat[t - lookback : t]                       # (lookback, time_dim)

            y = demand[t : t + horizon]                               # (horizon,)
            dec_t = time_feat[t : t + horizon]                        # (horizon, time_dim)

            # Decoder input = shifted right by 1:
            #   at step0 we provide last encoder value (or 0) as "previous demand"
            # This "anchors" the decoder to the last observed level.
            last_enc = enc_d[-1]
            dec_in = np.concatenate([[last_enc], y[:-1]], axis=0)      # (horizon,)

            enc_demand_list.append(enc_d[:, None])                     # add feature dim => (lookback, 1)
            enc_time_list.append(enc_t)
            dec_in_demand_list.append(dec_in[:, None])                 # (horizon, 1)
            dec_time_list.append(dec_t)
            y_list.append(y[:, None])                                  # (horizon, 1)

            sku_id_list.append(sku_map[sku])
            loc_id_list.append(loc_map[loc])

    return WindowedDataset(
        enc_demand=np.array(enc_demand_list, dtype=np.float32),
        enc_time=np.array(enc_time_list, dtype=np.float32),
        dec_in_demand=np.array(dec_in_demand_list, dtype=np.float32),
        dec_time=np.array(dec_time_list, dtype=np.float32),
        y=np.array(y_list, dtype=np.float32),
        sku_id=np.array(sku_id_list, dtype=np.int32),
        loc_id=np.array(loc_id_list, dtype=np.int32),
    )


def train_val_split(w: WindowedDataset, val_ratio: float = 0.2) -> Tuple[WindowedDataset, WindowedDataset]:
    """Simple random split for demo purposes."""
    n = w.y.shape[0]
    idx = np.arange(n)
    np.random.shuffle(idx)
    n_val = int(round(n * val_ratio))
    val_idx = idx[:n_val]
    trn_idx = idx[n_val:]

    def take(arr, ids):
        return arr[ids]

    trn = WindowedDataset(
        enc_demand=take(w.enc_demand, trn_idx),
        enc_time=take(w.enc_time, trn_idx),
        dec_in_demand=take(w.dec_in_demand, trn_idx),
        dec_time=take(w.dec_time, trn_idx),
        y=take(w.y, trn_idx),
        sku_id=take(w.sku_id, trn_idx),
        loc_id=take(w.loc_id, trn_idx),
    )
    val = WindowedDataset(
        enc_demand=take(w.enc_demand, val_idx),
        enc_time=take(w.enc_time, val_idx),
        dec_in_demand=take(w.dec_in_demand, val_idx),
        dec_time=take(w.dec_time, val_idx),
        y=take(w.y, val_idx),
        sku_id=take(w.sku_id, val_idx),
        loc_id=take(w.loc_id, val_idx),
    )
    return trn, val



In [7]:
# =============================================================================
# 3. MODEL: ENCODER-DECODER (TEACHER FORCING TRAINING GRAPH)
# =============================================================================

def build_seq2seq_model(
    n_skus: int,
    n_locations: int,
    time_dim: int = 3,
    embed_dim: int = EMBED_DIM,
    enc_units: int = ENCODER_UNITS,
    dec_units: int = DECODER_UNITS,
    dropout: float = DROPOUT,
) -> tf.keras.Model:
    """
    Encoder-Decoder with LSTM.

    Inputs:
      - encoder_demand: (B, lookback, 1)
      - encoder_time:   (B, lookback, time_dim)
      - decoder_demand: (B, horizon, 1)    teacher-forced "previous y"
      - decoder_time:   (B, horizon, time_dim)
      - sku_id:         (B,)
      - loc_id:         (B,)

    Output:
      - y_hat: (B, horizon, 1) predicted demand_scaled curve

    Why embeddings:
      - Cold-start SKUs: embeddings provide a learnable prior per SKU/Location.
      - It’s a pragmatic compromise when you do not have rich product attributes.
    """
    # ----- Inputs
    enc_demand_in = tf.keras.Input(shape=(None, 1), name="enc_demand")
    enc_time_in = tf.keras.Input(shape=(None, time_dim), name="enc_time")
    dec_demand_in = tf.keras.Input(shape=(None, 1), name="dec_demand_in")
    dec_time_in = tf.keras.Input(shape=(None, time_dim), name="dec_time")

    sku_id_in = tf.keras.Input(shape=(), dtype=tf.int32, name="sku_id")
    loc_id_in = tf.keras.Input(shape=(), dtype=tf.int32, name="loc_id")

    # ----- Embeddings
    sku_emb = tf.keras.layers.Embedding(n_skus, embed_dim, name="sku_emb")(sku_id_in)   # (B, embed_dim)
    loc_emb = tf.keras.layers.Embedding(n_locations, embed_dim, name="loc_emb")(loc_id_in)

    # Combine embeddings -> one context vector
    context = tf.keras.layers.Concatenate(name="id_context")([sku_emb, loc_emb])        # (B, 2*embed_dim)
    context = tf.keras.layers.Dropout(dropout)(context)

    # Repeat context across time for encoder and decoder
    def repeat_context(x, steps_tensor):
        # steps_tensor is a sequence input; we repeat context to match its time dimension
        steps = tf.shape(steps_tensor)[1]
        return tf.tile(x[:, None, :], multiples=[1, steps, 1])

    enc_ctx = tf.keras.layers.Lambda(lambda t: repeat_context(t[0], t[1]), name="enc_ctx")([context, enc_demand_in])
    dec_ctx = tf.keras.layers.Lambda(lambda t: repeat_context(t[0], t[1]), name="dec_ctx")([context, dec_demand_in])

    # ----- Encoder
    enc_x = tf.keras.layers.Concatenate(name="enc_concat")([enc_demand_in, enc_time_in, enc_ctx])
    enc_x = tf.keras.layers.LayerNormalization()(enc_x)
    enc_lstm = tf.keras.layers.LSTM(
        enc_units,
        return_state=True,
        name="encoder_lstm",
        dropout=dropout,
        recurrent_dropout=0.0,  # recurrent dropout can be slower/unstable; keep simple
    )
    _, enc_h, enc_c = enc_lstm(enc_x)

    # ----- Decoder (teacher forcing)
    dec_x = tf.keras.layers.Concatenate(name="dec_concat")([dec_demand_in, dec_time_in, dec_ctx])
    dec_x = tf.keras.layers.LayerNormalization()(dec_x)
    dec_lstm = tf.keras.layers.LSTM(
        dec_units,
        return_sequences=True,
        return_state=True,
        name="decoder_lstm",
        dropout=dropout,
        recurrent_dropout=0.0,
    )
    dec_seq, _, _ = dec_lstm(dec_x, initial_state=[enc_h, enc_c])

    # Output projection per time step
    y_hat = tf.keras.layers.Dense(1, name="y_hat")(dec_seq)

    model = tf.keras.Model(
        inputs={
            "enc_demand": enc_demand_in,
            "enc_time": enc_time_in,
            "dec_demand_in": dec_demand_in,
            "dec_time": dec_time_in,
            "sku_id": sku_id_in,
            "loc_id": loc_id_in,
        },
        outputs=y_hat,
        name="seq2seq_demand_forecaster",
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss=tf.keras.losses.Huber(),  # robust to spikes/promos
        metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae")],
    )
    return model


In [8]:
# =============================================================================
# 4. FORECASTING MODES: DIRECT (ONE-SHOT) VS RECURSIVE/ROLLING (AUTO-REGRESSIVE)
# =============================================================================

def direct_forecast(
    model: tf.keras.Model,
    enc_demand: np.ndarray,
    enc_time: np.ndarray,
    dec_time: np.ndarray,
    sku_id: np.ndarray,
    loc_id: np.ndarray,
) -> np.ndarray:
    """
    DIRECT multi-step forecast:
    - Provide decoder inputs (previous y) as a constant seed repeated, OR
      use last encoder value and then zeros (naive).
    - Model outputs full curve in one forward pass.

    Why it’s called "direct":
    - You predict all horizons jointly without feeding predictions back.

    Trade-off:
    - Pros: avoids error accumulation.
    - Cons: decoder inputs are not "true previous y" at inference.
    """
    # Seed: last observed demand from encoder (per sample)
    last_y = enc_demand[:, -1:, :]                             # (B, 1, 1)
    horizon = dec_time.shape[1]

    # Build decoder input: last_y then zeros (simple, common baseline)
    zeros = np.zeros((enc_demand.shape[0], horizon - 1, 1), dtype=np.float32)
    dec_in = np.concatenate([last_y, zeros], axis=1)           # (B, H, 1)

    y_hat = model.predict(
        {
            "enc_demand": enc_demand,
            "enc_time": enc_time,
            "dec_demand_in": dec_in,
            "dec_time": dec_time,
            "sku_id": sku_id,
            "loc_id": loc_id,
        },
        verbose=0,
    )
    return y_hat


def rolling_forecast(
    model: tf.keras.Model,
    enc_demand: np.ndarray,
    enc_time: np.ndarray,
    dec_time: np.ndarray,
    sku_id: np.ndarray,
    loc_id: np.ndarray,
) -> np.ndarray:
    """
    ROLLING / RECURSIVE multi-step forecast:
    - Predict step 1
    - Feed prediction back as previous y for step 2
    - Repeat until horizon H

    Why people use it:
    - Closer to how teacher forcing is trained (decoder sees previous y values).
    - Works well when short-horizon accuracy is strong and dynamics are stable.

    Risk:
    - Error accumulation: small bias at step 1 can compound by step H.
    """
    b = enc_demand.shape[0]
    h = dec_time.shape[1]

    last_y = enc_demand[:, -1, :]                              # (B, 1)
    preds = []

    prev_y = last_y.copy()                                     # (B, 1)
    for t in range(h):
        # Construct decoder input for this step: shape (B, 1, 1)
        dec_in_step = prev_y[:, None, :]                       # (B, 1, 1)
        dec_time_step = dec_time[:, t:t+1, :]                  # (B, 1, time_dim)

        y_hat_step = model.predict(
            {
                "enc_demand": enc_demand,
                "enc_time": enc_time,
                "dec_demand_in": dec_in_step,
                "dec_time": dec_time_step,
                "sku_id": sku_id,
                "loc_id": loc_id,
            },
            verbose=0,
        )                                                      # (B, 1, 1)

        preds.append(y_hat_step)
        prev_y = y_hat_step[:, 0, :]                           # feed back (B, 1)

    return np.concatenate(preds, axis=1)                        # (B, H, 1)


In [9]:
# =============================================================================
# 5. TRANSLATING FORECASTS INTO REPLENISHMENT DECISIONS
# =============================================================================

def z_value(service_level: float) -> float:
    """
    Approximate z-score for a service level using inverse error function.

    Why this matters:
    - For a simple safety stock approximation: SS = z * sigma_LT
    - In production, you'd use scipy.stats.norm.ppf; we avoid extra deps here.
    """
    # Convert service level to a normal quantile approximation
    # Using approximation via inverse erf: ppf(p) = sqrt(2) * erfinv(2p - 1)
    return math.sqrt(2) * float(tf.math.erfinv(2.0 * service_level - 1.0).numpy())


def replenishment_from_forecast(
    forecast_daily: np.ndarray,
    on_hand: float,
    on_order: float,
    lead_time_days: int,
    service_level: float = SERVICE_LEVEL,
    min_order_qty: int = MIN_ORDER_QTY,
) -> Dict[str, float]:
    """
    Convert an H-day forecast curve into a simple reorder suggestion.

    Policy (simplified continuous review style):
      Inventory Position (IP) = on_hand + on_order
      Lead-time demand (LTD) = sum(forecast over lead_time_days)
      Safety stock (SS) approximated from forecast volatility
      Reorder point (ROP) = LTD + SS
      Order qty = max(0, ROP - IP), optionally rounded / MOQ

    Why this is useful pedagogically:
    - Forecasts are not the end goal; decisions are.
    - Students see how multi-step forecasts map to inventory parameters.
    """
    lead_time_days = int(max(1, lead_time_days))
    ltd = float(np.sum(forecast_daily[:lead_time_days]))

    # Approximate sigma over lead time using daily std * sqrt(L)
    daily_sigma = float(np.std(forecast_daily[:lead_time_days], ddof=0))
    sigma_lt = daily_sigma * math.sqrt(lead_time_days)

    z = z_value(service_level)
    safety_stock = z * sigma_lt
    reorder_point = ltd + safety_stock

    inventory_position = float(on_hand + on_order)
    raw_order = max(0.0, reorder_point - inventory_position)

    # Apply MOQ / rounding if desired (keep simple for course)
    order_qty = float(max(raw_order, float(min_order_qty))) if raw_order > 0 else 0.0

    return {
        "lead_time_demand": ltd,
        "daily_sigma": daily_sigma,
        "safety_stock": float(safety_stock),
        "reorder_point": float(reorder_point),
        "inventory_position": inventory_position,
        "order_qty_suggested": order_qty,
    }


In [10]:
# =============================================================================
# 6. PROGRESSIVE EXAMPLES (FROM SIMPLE TO REALISTIC)
# =============================================================================

def example_1_minimal_seq2seq_shapes() -> None:
    """
    Example 1: Minimal sanity check of shapes only.

    Why:
    - Learners often get stuck on 3D tensors (B, T, F).
    - This example isolates the mechanics without business complexity.
    """
    b, lookback, horizon = 4, 7, 3
    time_dim = 3

    enc_demand = np.random.randn(b, lookback, 1).astype(np.float32)
    enc_time = np.random.randn(b, lookback, time_dim).astype(np.float32)
    dec_time = np.random.randn(b, horizon, time_dim).astype(np.float32)

    sku_id = np.random.randint(0, 10, size=(b,), dtype=np.int32)
    loc_id = np.random.randint(0, 2, size=(b,), dtype=np.int32)

    model = build_seq2seq_model(n_skus=10, n_locations=2, time_dim=time_dim)
    y_hat = direct_forecast(model, enc_demand, enc_time, dec_time, sku_id, loc_id)

    print("Example 1 shapes")
    print("enc_demand:", enc_demand.shape)
    print("dec_time:  ", dec_time.shape)
    print("y_hat:     ", y_hat.shape)
    print("-" * 60)


def example_2_train_on_simulated_panel() -> Tuple[tf.keras.Model, pd.DataFrame, Dict, Dict, Dict]:
    """
    Example 2: End-to-end training on simulated SKU-location panel.

    What you learn here:
    - How to create teacher-forced decoder inputs
    - How seq2seq can output full demand curves
    - How cold-start SKUs are represented
    """
    df = simulate_sku_location_demand(n_skus=80, n_locations=4, n_days=420, cold_start_ratio=0.25)
    df = add_time_features(df)

    sku_map, loc_map = build_id_maps(df)

    # Fit scaler on full data for demo simplicity
    # In production, fit on TRAIN period only to avoid leakage.
    stats = fit_scaler_per_series(df)
    df_scaled = apply_scaler(df, stats)

    w = make_seq2seq_windows(df_scaled, sku_map, loc_map, LOOKBACK_DAYS, FORECAST_HORIZON, step=2)
    trn, val = train_val_split(w, val_ratio=0.2)

    model = build_seq2seq_model(n_skus=len(sku_map), n_locations=len(loc_map), time_dim=3)

    # Build tf.data (preferred for performance and cleanliness)
    def to_ds(ws: WindowedDataset, shuffle: bool) -> tf.data.Dataset:
        x = {
            "enc_demand": ws.enc_demand,
            "enc_time": ws.enc_time,
            "dec_demand_in": ws.dec_in_demand,
            "dec_time": ws.dec_time,
            "sku_id": ws.sku_id,
            "loc_id": ws.loc_id,
        }
        y = ws.y
        ds = tf.data.Dataset.from_tensor_slices((x, y))
        if shuffle:
            ds = ds.shuffle(buffer_size=min(50_000, y.shape[0]), seed=RANDOM_SEED)
        return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    trn_ds = to_ds(trn, shuffle=True)
    val_ds = to_ds(val, shuffle=False)

    model.fit(trn_ds, validation_data=val_ds, epochs=EPOCHS, verbose=1)

    artifacts = {
        "df": df_scaled,
        "stats": stats,
        "sku_map": sku_map,
        "loc_map": loc_map,
    }
    return model, df_scaled, stats, sku_map, loc_map


def example_3_compare_direct_vs_recursive(
    model: tf.keras.Model,
    df_scaled: pd.DataFrame,
    stats: Dict[Tuple[str, str], Tuple[float, float]],
    sku_map: Dict[str, int],
    loc_map: Dict[str, int],
) -> None:
    """
    Example 3: Compare DIRECT vs ROLLING forecasts for a single (SKU,LOC).

    What to look for:
    - Direct often has smoother curves
    - Rolling can track momentum but may drift if biased early
    """
    # Pick a random series
    sku = np.random.choice(list(sku_map.keys()))
    loc = np.random.choice(list(loc_map.keys()))

    g = df_scaled[(df_scaled[COL_SKU] == sku) & (df_scaled[COL_LOC] == loc)].sort_values(COL_DATE)
    g = g.reset_index(drop=True)

    # Choose a forecast origin near the end
    t = len(g) - FORECAST_HORIZON - 1
    t = max(LOOKBACK_DAYS, t)

    enc = g.iloc[t - LOOKBACK_DAYS : t]
    fut = g.iloc[t : t + FORECAST_HORIZON]

    enc_demand = enc["demand_scaled"].to_numpy(np.float32)[None, :, None]
    enc_time = enc[["dow", "month", "is_weekend"]].to_numpy(np.float32)[None, :, :]
    dec_time = fut[["dow", "month", "is_weekend"]].to_numpy(np.float32)[None, :, :]

    sku_id = np.array([sku_map[sku]], dtype=np.int32)
    loc_id = np.array([loc_map[loc]], dtype=np.int32)

    y_hat_direct = direct_forecast(model, enc_demand, enc_time, dec_time, sku_id, loc_id)[0, :, 0]
    y_hat_roll = rolling_forecast(model, enc_demand, enc_time, dec_time, sku_id, loc_id)[0, :, 0]
    y_true = fut["demand_scaled"].to_numpy(np.float32)

    # Inverse scaling back to demand units
    mu, sigma = stats[(sku, loc)]
    y_direct = y_hat_direct * sigma + mu
    y_roll = y_hat_roll * sigma + mu
    y_true_u = y_true * sigma + mu

    mae_direct = float(np.mean(np.abs(y_direct - y_true_u)))
    mae_roll = float(np.mean(np.abs(y_roll - y_true_u)))

    print("Example 3: Direct vs Rolling")
    print(f"Series: {sku} @ {loc}")
    print(f"MAE Direct : {mae_direct:.2f}")
    print(f"MAE Rolling: {mae_roll:.2f}")
    print("First 5 days (true / direct / rolling):")
    for i in range(min(5, FORECAST_HORIZON)):
        print(f"  D+{i+1:02d}: {y_true_u[i]:7.2f} / {y_direct[i]:7.2f} / {y_roll[i]:7.2f}")
    print("-" * 60)


def example_4_forecast_to_replenishment(
    model: tf.keras.Model,
    df_scaled: pd.DataFrame,
    stats: Dict[Tuple[str, str], Tuple[float, float]],
    sku_map: Dict[str, int],
    loc_map: Dict[str, int],
) -> None:
    """
    Example 4: Convert a forecast curve into a reorder suggestion.

    This connects forecasting to a real SCM decision.
    """
    sku = np.random.choice(list(sku_map.keys()))
    loc = np.random.choice(list(loc_map.keys()))

    g = df_scaled[(df_scaled[COL_SKU] == sku) & (df_scaled[COL_LOC] == loc)].sort_values(COL_DATE)
    g = g.reset_index(drop=True)

    t = len(g) - FORECAST_HORIZON - 1
    t = max(LOOKBACK_DAYS, t)

    enc = g.iloc[t - LOOKBACK_DAYS : t]
    fut = g.iloc[t : t + FORECAST_HORIZON]

    enc_demand = enc["demand_scaled"].to_numpy(np.float32)[None, :, None]
    enc_time = enc[["dow", "month", "is_weekend"]].to_numpy(np.float32)[None, :, :]
    dec_time = fut[["dow", "month", "is_weekend"]].to_numpy(np.float32)[None, :, :]

    sku_id = np.array([sku_map[sku]], dtype=np.int32)
    loc_id = np.array([loc_map[loc]], dtype=np.int32)

    # Use direct forecast for stability in this demo
    y_hat_scaled = direct_forecast(model, enc_demand, enc_time, dec_time, sku_id, loc_id)[0, :, 0]

    # Back to demand units
    mu, sigma = stats[(sku, loc)]
    forecast_units = (y_hat_scaled * sigma + mu).clip(min=0.0)

    # Example inventory situation (in real life: from ERP/WMS)
    on_hand = float(np.random.uniform(0, 200))
    on_order = float(np.random.uniform(0, 150))
    lead_time_days = int(np.random.choice([3, 5, 7, 10, 14]))

    decision = replenishment_from_forecast(
        forecast_daily=forecast_units,
        on_hand=on_hand,
        on_order=on_order,
        lead_time_days=lead_time_days,
        service_level=SERVICE_LEVEL,
        min_order_qty=MIN_ORDER_QTY,
    )

    print("Example 4: Forecast -> Replenishment")
    print(f"Series: {sku} @ {loc}")
    print(f"On hand: {on_hand:.1f} | On order: {on_order:.1f} | Lead time: {lead_time_days} days")
    print(f"Lead-time demand: {decision['lead_time_demand']:.2f}")
    print(f"Safety stock:     {decision['safety_stock']:.2f} (service level={SERVICE_LEVEL})")
    print(f"Reorder point:    {decision['reorder_point']:.2f}")
    print(f"Inv position:     {decision['inventory_position']:.2f}")
    print(f"Order suggested:  {decision['order_qty_suggested']:.2f}")
    print("-" * 60)

In [ ]:
# =============================================================================
# 7. TINY "TRY YOURSELF" TASKS (WITH OPTIONAL SOLUTIONS)
# =============================================================================

# TODO 1:
# Change FORECAST_HORIZON from 14 to 28 and retrain.
# Question: Does MAE improve or degrade? Why might longer horizons be harder?

# TODO 2:
# Change SERVICE_LEVEL from 0.95 to 0.99 and re-run Example 4.
# Question: How does safety stock change? Why is the relationship non-linear?

# TODO 3:
# Switch loss from Huber() to MeanSquaredError().
# Question: What happens with promo spikes? Which loss is more robust?

# --- Optional Solution Hints (keep short, let learners experiment) ---
# - Longer horizons often degrade because uncertainty grows and the model must
#   learn longer-range seasonality/trend, plus cold-start becomes more severe.
# - Service level maps to z-score; going 0.95 -> 0.99 increases z materially.
# - MSE penalizes large errors strongly, which can overreact to promotions.

In [11]:
# =============================================================================
# 8. BUILT-IN CHECKS / DEMO RUNNER
# =============================================================================

def main() -> None:
    # Example 1: minimal shapes sanity check
    example_1_minimal_seq2seq_shapes()

    # Example 2: train on simulated panel (this is the "core" lesson)
    model, df_scaled, stats, sku_map, loc_map = example_2_train_on_simulated_panel()

    # Example 3: compare recursive vs direct forecasting
    example_3_compare_direct_vs_recursive(model, df_scaled, stats, sku_map, loc_map)

    # Example 4: show replenishment decision from forecast curve
    example_4_forecast_to_replenishment(model, df_scaled, stats, sku_map, loc_map)


if __name__ == "__main__":
    main()


Example 1 shapes
enc_demand: (4, 7, 1)
dec_time:   (4, 3, 3)
y_hat:      (4, 3, 1)
------------------------------------------------------------
Epoch 1/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 14s 47ms/step - loss: 0.2738 - mae: 0.5921 - val_loss: 0.2354 - val_mae: 0.5316
Epoch 2/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - loss: 0.2347 - mae: 0.5298 - val_loss: 0.2253 - val_mae: 0.5115
Epoch 3/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 9s 46ms/step - loss: 0.2266 - mae: 0.5151 - val_loss: 0.2225 - val_mae: 0.5053
Epoch 4/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - loss: 0.2236 - mae: 0.5095 - val_loss: 0.2242 - val_mae: 0.5046
Epoch 5/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 0.2222 - mae: 0.5068 - val_loss: 0.2192 - val_mae: 0.5020
Epoch 6/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 0.2211 - mae: 0.5043 - val_loss: 0.2193 - val_mae: 0.4986
Epoch 7/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 0.2202 - mae: 0.5026 - val_loss: 0.2191 - val_mae: 0.4980
Epoch 8/10
190/190 ━━